In [1]:
import matplotlib as mpl
mpl.use("pgf")
import numpy as np
mm_to_in = 1/25.4
pgf_with_pdflatex = {
    "pgf.texsystem": "pdflatex",
    "pgf.preamble": [
         r"\usepackage{mathtools}",
         ],
    "text.usetex": True,
    "figure.figsize": [190*mm_to_in, 3.0],
    "axes.labelsize": 10,
    "axes.labelpad": 5.0,
    "font.size": 12,
    "font.family": "serif",
    "legend.fontsize": 8,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "lines.linewidth": 1.5,
    "xtick.major.size": 5,
    "xtick.major.width": 1,
    "xtick.minor.size": 2.5,
    "xtick.minor.width": 1,
    "ytick.major.size": 5,
    "ytick.major.width": 1,
    "ytick.minor.size": 2.5,
    "ytick.minor.width": 1,
}
mpl.rcParams.update(pgf_with_pdflatex)
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, AutoMinorLocator
from matplotlib.lines import Line2D
from scipy.interpolate import UnivariateSpline
from matplotlib.patches import Rectangle
base = Path('constant-volume-comparison')

In [2]:
names = [('15.0E5', '0.25'), ('15.0E5', '0.5'), ('15.0E5', '1.0'), ('15.0E5', '2.0'), ('30.0E5', '0.25'), ('30.0E5', '0.5'), ('30.0E5', '1.0'), ('30.0E5', '2.0')]
det = pd.read_csv(base/'det-conv-sims.csv', header=None, names=names)
det_15 = det['15.0E5']
det_30 = det['30.0E5']
rmg = pd.read_csv(base/'rmg-conv-sims.csv', header=None, names=names)
rmg_15 = rmg['15.0E5']
rmg_30 = rmg['30.0E5']

In [3]:
names = [('15.0E5', '0.25'), ('30.0E5', '0.25'), ('15.0E5', '0.5'), ('30.0E5', '0.5'), ('15.0E5', '1.0'), ('30.0E5', '1.0'), ('15.0E5', '2.0'), ('30.0E5', '2.0')]
det = pd.read_csv(base/'det-conv-sims-400.csv', header=None, names=names)
det_15_ar = det['15.0E5']
det_30_ar = det['30.0E5']
rmg = pd.read_csv(base/'rmg-conv-sims-400.csv', header=None, names=names)
rmg_15_ar = rmg['15.0E5']
rmg_30_ar = rmg['30.0E5']

In [4]:
data = pd.read_csv('simulation-comparison/complete-data-set-w-sims.csv')

phi_200_15_bar = data[(data['Equivalence Ratio'] == 2.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1.0E-2))][::-1]
phi_100_15_bar = data[(data['Equivalence Ratio'] == 1.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1.0E-2))][::-1]
phi_100_30_bar = data[(data['Equivalence Ratio'] == 1.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1.0E-2))][::-1]
phi_050_15_bar = data[(data['Equivalence Ratio'] == 0.50) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1.0E-1))][::-1]
phi_050_30_bar = data[(data['Equivalence Ratio'] == 0.50) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1.0E-2))][::-1]
phi_025_15_bar = data[(data['Equivalence Ratio'] == 0.25) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1.0E-2))][::-1]
phi_025_30_bar = data[(data['Equivalence Ratio'] == 0.25) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1.0E-2))][::-1]

In [12]:
fig, (ax_15_bar, ax_30_bar) = plt.subplots(ncols=2)

ax_15_bar.set_yscale('log')
ax_30_bar.set_yscale('log')

# plot_opts = {'markersize': 7, 'fmt': '', 'elinewidth': 1.5, 'capthick': 1.5, 'linestyle': 'None'}

colors = {'0.25': 'C0', '0.5': 'C1', '1.0': 'C2', '2.0': 'C3'}
phi_025_color = colors['0.25']
phi_050_color = colors['0.5']
phi_100_color = colors['1.0']
phi_200_color = colors['2.0']

# Set the formatting of the y tick labels
ax_15_bar.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax_30_bar.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax_15_bar.xaxis.set_minor_locator(AutoMinorLocator(4))
ax_30_bar.xaxis.set_minor_locator(AutoMinorLocator(4))

# Set x limits
ax_15_bar.set_xlim(0.94, 1.55)
ax_15_bar.set_ylim(0.7, 100.0)
ax_30_bar.set_xlim(0.92, 1.55)


# Set the axis labels
fig.text(0.0, 0.5, 'Overall Ignition Delay, ms', verticalalignment='center', rotation='vertical')
fig.text(0.5, 0.01, '$1000/T_C$, 1/K', horizontalalignment='center')

# Set the a) b) figure labels
ax_15_bar.text(0.6, 0.05, r'a) $15\ \text{bar}$', transform=ax_15_bar.transAxes)
ax_30_bar.text(0.6, 0.05, r'b) $30\ \text{bar}$', transform=ax_30_bar.transAxes)

# Create the temperature axes on the top
def convert_inv_temp(temps):
    """Convert a list of temperatures to inverse temperature"""
    return [1000.0/temp for temp in temps]

ax_15_temp = ax_15_bar.twiny()
ax_30_temp = ax_30_bar.twiny()

# Set the major tick marks in the temperature scale and convert to inverse scale
major_temps = np.arange(1100, 600, -100)
major_ticks = convert_inv_temp(major_temps)

# Set the interval for the minor ticks and compute the minor ticks
minor_interval = 20
minor_ticks = []
for maj in major_temps:
    minor_ticks.extend(convert_inv_temp([maj - i*minor_interval for i in range(5)]))

# Set the ticks on the axis. Note that the limit setting must be present and must be after setting the ticks
# so that the scale is correct
ax_15_temp.set_xticks(major_ticks)
ax_15_temp.set_xticks(minor_ticks, minor=True)
ax_15_temp.set_xticklabels(['{:d} K'.format(temp) for temp in major_temps])
ax_15_temp.set_xlim(ax_15_bar.get_xlim())
ax_30_temp.set_xticks(major_ticks)
ax_30_temp.set_xticks(minor_ticks, minor=True)
ax_30_temp.set_xticklabels(['{:d} K'.format(temp) for temp in major_temps])
ax_30_temp.set_xlim(ax_30_bar.get_xlim())

plot_opts_sim_line_15 = {}
plot_opts_sim_rcm = {'linestyle': 'solid'}
plot_opts_sim_line_30 = {}
plot_opts_sim_conv = {'linestyle': 'dotted'}

for p in ['0.25', '0.5', '1.0', '2.0']:
#     ax_15_bar.plot(1000.0/det_15.index, det_15[p]*1000, color=colors[p], label=r'$\phi = {}$'.format(p), **plot_opts_sim_conv)
#     ax_30_bar.plot(1000.0/det_30.index, det_30[p]*1000, color=colors[p], label=r'$\phi = {}$'.format(p), **plot_opts_sim_conv)
    ax_15_bar.plot(1000.0/det_15_ar.index, det_15_ar[p]*1000, color=colors[p], label=r'$\phi = {}$'.format(p), **plot_opts_sim_conv)
    ax_30_bar.plot(1000.0/det_30_ar.index, det_30_ar[p]*1000, color=colors[p], label=r'$\phi = {}$'.format(p), **plot_opts_sim_conv)

    #     ax_15_bar.plot(1000.0/rmg_15.index, rmg_15[p]*1000, color=colors[p], **plot_opts_sim_rmg)

In [13]:
phi_025_15_rmg = phi_025_15_bar.dropna(subset=['chem rmg (ms)']).sort_values(by='1000/Tc (1/K)')
phi_025_15_det = phi_025_15_bar.dropna(subset=['Detailed Model (ms)']).sort_values(by='1000/Tc (1/K)')
phi_025_15_rmg_temps = np.linspace(phi_025_15_rmg['1000/Tc (1/K)'].iloc[0], phi_025_15_rmg['1000/Tc (1/K)'].iloc[-1])
phi_025_15_det_temps = np.linspace(phi_025_15_det['1000/Tc (1/K)'].iloc[0], phi_025_15_det['1000/Tc (1/K)'].iloc[-1])
# ax_15_bar.plot(phi_025_15_rmg['1000/Tc (1/K)'], phi_025_15_rmg['chem rmg (ms)'], **plot_opts_sim_mark_15, **plot_opts_sim_rmg)
# spl = UnivariateSpline(phi_025_15_rmg['1000/Tc (1/K)'], np.log(phi_025_15_rmg['chem rmg (ms)']))
# ax_15_bar.plot(phi_025_15_rmg_temps, np.exp(spl(phi_025_15_rmg_temps)), color=phi_025_color, **plot_opts_sim_rmg)
# ax_15_bar.plot(phi_025_15_det['1000/Tc (1/K)'], phi_025_15_det['Detailed Model (ms)'], **plot_opts_sim_mark_15, **plot_opts_sim_det)
spl = UnivariateSpline(phi_025_15_det['1000/Tc (1/K)'], np.log(phi_025_15_det['Detailed Model (ms)']))
ax_15_bar.plot(phi_025_15_det_temps, np.exp(spl(phi_025_15_det_temps)), color=phi_025_color, **plot_opts_sim_rcm);

In [14]:
phi_050_15_rmg = phi_050_15_bar.dropna(subset=['chem rmg (ms)']).sort_values(by='1000/Tc (1/K)')
phi_050_15_det = phi_050_15_bar.dropna(subset=['Detailed Model (ms)']).sort_values(by='1000/Tc (1/K)')
phi_050_15_rmg_temps = np.linspace(phi_050_15_rmg['1000/Tc (1/K)'].iloc[0], phi_050_15_rmg['1000/Tc (1/K)'].iloc[-1])
phi_050_15_det_temps = np.linspace(phi_050_15_det['1000/Tc (1/K)'].iloc[0], phi_050_15_det['1000/Tc (1/K)'].iloc[-1])
# ax_15_bar.plot(phi_050_15_rmg['1000/Tc (1/K)'], phi_050_15_rmg['chem rmg (ms)'], **plot_opts_sim_mark_15, **plot_opts_sim_rmg)
# spl = UnivariateSpline(phi_050_15_rmg['1000/Tc (1/K)'], np.log(phi_050_15_rmg['chem rmg (ms)']))
# ax_15_bar.plot(phi_050_15_rmg_temps, np.exp(spl(phi_050_15_rmg_temps)), color=phi_050_color, **plot_opts_sim_rmg)
# ax_15_bar.plot(phi_050_15_det['1000/Tc (1/K)'], phi_050_15_det['Detailed Model (ms)'], **plot_opts_sim_mark_15, **plot_opts_sim_det)
spl = UnivariateSpline(phi_050_15_det['1000/Tc (1/K)'], np.log(phi_050_15_det['Detailed Model (ms)']))
ax_15_bar.plot(phi_050_15_det_temps, np.exp(spl(phi_050_15_det_temps)), color=phi_050_color, **plot_opts_sim_rcm);

In [15]:
# phi_100_15_rmg = phi_100_15_bar.dropna(subset=['chem rmg (ms)']).sort_values(by='1000/Tc (1/K)')
# phi_100_15_det = phi_100_15_bar.dropna(subset=['Detailed Model (ms)']).sort_values(by='1000/Tc (1/K)')
phi_100_30_rmg = phi_100_30_bar.dropna(subset=['chem rmg (ms)']).sort_values(by='1000/Tc (1/K)')
phi_100_30_det = phi_100_30_bar.dropna(subset=['Detailed Model (ms)']).sort_values(by='1000/Tc (1/K)')
phi_100_30_rmg_temps = np.linspace(phi_100_30_rmg['1000/Tc (1/K)'].iloc[0], phi_100_30_rmg['1000/Tc (1/K)'].iloc[-1])
phi_100_30_det_temps = np.linspace(phi_100_30_det['1000/Tc (1/K)'].iloc[0], phi_100_30_det['1000/Tc (1/K)'].iloc[-1])
# ax_30_bar.plot(phi_100_30_rmg['1000/Tc (1/K)'], phi_100_30_rmg['chem rmg (ms)'], **plot_opts_sim_mark_30, **plot_opts_sim_rmg)
# spl = UnivariateSpline(phi_100_30_rmg['1000/Tc (1/K)'], np.log(phi_100_30_rmg['chem rmg (ms)']))
# ax_30_bar.plot(phi_100_30_rmg_temps, np.exp(spl(phi_100_30_rmg_temps)), color=phi_100_color, **plot_opts_sim_rmg)
# ax_30_bar.plot(phi_100_30_det['1000/Tc (1/K)'], phi_100_30_det['Detailed Model (ms)'], **plot_opts_sim_mark_30, **plot_opts_sim_det)
spl = UnivariateSpline(phi_100_30_det['1000/Tc (1/K)'], np.log(phi_100_30_det['Detailed Model (ms)']))
ax_30_bar.plot(phi_100_30_det_temps, np.exp(spl(phi_100_30_det_temps)), color=phi_100_color, **plot_opts_sim_rcm);

In [16]:
phi_200_15_rmg = phi_200_15_bar.dropna(subset=['chem rmg (ms)']).sort_values(by='1000/Tc (1/K)')
phi_200_15_det = phi_200_15_bar.dropna(subset=['Detailed Model (ms)']).sort_values(by='1000/Tc (1/K)')
phi_200_15_rmg_temps = np.linspace(phi_200_15_rmg['1000/Tc (1/K)'].iloc[0], phi_200_15_rmg['1000/Tc (1/K)'].iloc[-1])
phi_200_15_det_temps = np.linspace(phi_200_15_det['1000/Tc (1/K)'].iloc[0], phi_200_15_det['1000/Tc (1/K)'].iloc[-1])
# ax_15_bar.plot(phi_200_15_rmg['1000/Tc (1/K)'], phi_200_15_rmg['chem rmg (ms)'], **plot_opts_sim_mark_15, **plot_opts_sim_rmg)
# spl = UnivariateSpline(phi_200_15_rmg['1000/Tc (1/K)'], np.log(phi_200_15_rmg['chem rmg (ms)']))
# ax_15_bar.plot(phi_200_15_rmg_temps, np.exp(spl(phi_200_15_rmg_temps)), color=phi_200_color, **plot_opts_sim_rmg)
# ax_15_bar.plot(phi_200_15_det['1000/Tc (1/K)'], phi_200_15_det['Detailed Model (ms)'], **plot_opts_sim_mark_15, **plot_opts_sim_det)
spl = UnivariateSpline(phi_200_15_det['1000/Tc (1/K)'], np.log(phi_200_15_det['Detailed Model (ms)']))
ax_15_bar.plot(phi_200_15_det_temps, np.exp(spl(phi_200_15_det_temps)), color=phi_200_color, **plot_opts_sim_rcm);

In [17]:
# Create the legend, removing the error bars
_, labels = ax_15_bar.get_legend_handles_labels()
# handles = handles[::2]
# labels = labels[::2]

rcm_sim = Line2D([], [], color='black', **plot_opts_sim_conv)
conv_sim = Line2D([], [], color='black', **plot_opts_sim_rcm)
handles = []

for p in ['0.25', '0.5', '1.0', '2.0']:
    handles.append(Rectangle([0, 0], width=1.0, height=1.0, fill=True, color=colors[p]))

handles.extend([rcm_sim, conv_sim])
labels.extend(["Adiabatic Const. Volume", "RCM"])


fig.legend(handles, labels, loc='upper center', numpoints=1, frameon=True, handletextpad=0.5, handlelength=1.25, ncol=6, columnspacing=1.25, borderaxespad=0.0);

In [18]:
fig.tight_layout()
fig.subplots_adjust(top=0.84)
# fig.savefig('constant-volume-comparison.pgf')
fig.savefig('constant-volume-comparison.pdf')